In [1]:
import pandas as pd
ipl_2024 = pd.read_csv('ipl_2024_matches.csv')
ipl_summary = pd.DataFrame()

ipl_summary['Match_Number'] = ipl_2024['match_no']
ipl_summary['Team1'] = ipl_2024['team1']
ipl_summary['Team2'] = ipl_2024['team2']
ipl_summary['Toss_Winner'] = ipl_2024['toss_winner']
ipl_summary['Toss_Decision'] = ipl_2024['toss_decision']
ipl_summary['Innings1_Score'] = ipl_2024['innings1_score'] 
ipl_summary['Innings2_Score'] = ipl_2024['innings2_score'] 
ipl_summary['Winning_Team'] = ipl_2024['winning_team']
ipl_summary['Margin'] = ipl_2024['margin']

ipl_summary

,Match_Number,Team1,Team2,Toss_Winner,Toss_Decision,Innings1_Score,Innings2_Score,Winning_Team,Margin
0,1,RCB,CSK,RCB,bat,173.0,176.0,CSK,6.0
1,2,DC,PBKS,PBKS,field,174.0,177.0,PBKS,4.0
2,3,KKR,SRH,SRH,field,208.0,204.0,KKR,4.0
3,4,RR,LSG,RR,bat,193.0,173.0,RR,20.0
4,5,GT,MI,MI,field,168.0,162.0,GT,6.0
...,...,...,...,...,...,...,...,...,...
69,70,RR,KKR,KK,field,NaN,NaN,NaN,NaN
70,71,SRH,KKR,SRH,bat,159.0,164.0,KKR,8.0
71,72,RCB,RR,RR,field,172.0,174.0,RR,4.0
72,73,SRH,RR,RR,field,175.0,139.0,SRH,36.0


In [2]:
from numpy import random
random.seed(0)
points_table = pd.DataFrame(columns=['Team', 'Points', 'Score_Difference'])

for index, row in ipl_summary.iterrows():
    team1 = row['Team1']
    team2 = row['Team2']
    
    score1 = random.randint(50, 300)  
    score2 = random.randint(50, 300) 

    if score1 > score2:
        winner = team1
        loser = team2
    elif score2 > score1:
        winner = team2
        loser = team1
    else:
        winner = 'Tie'
        loser = 'Tie'
    
    if winner != 'Tie':
        if winner in points_table['Team'].values:
            points_table.loc[points_table['Team'] == winner, 'Points'] += 2
            points_table.loc[points_table['Team'] == winner, 'Score_Difference'] += (score1 - score2) if winner == team1 else (score2 - score1)
        else:
            points_table = pd.concat([points_table, pd.DataFrame({'Team': [winner], 'Points': [2], 'Score_Difference': (score1 - score2) if winner == team1 else (score2 - score1)})], ignore_index=True)
      
        if loser in points_table['Team'].values:
            points_table.loc[points_table['Team'] == loser, 'Score_Difference'] += (score2 - score1) if loser == team2 else (score1 - score2)
        else:
            points_table = pd.concat([points_table, pd.DataFrame({'Team': [loser], 'Points': [0], 'Score_Difference': (score2 - score1) if loser == team2 else (score1 - score2)})], ignore_index=True)
    else:
        if team1 in points_table['Team'].values:
            points_table.loc[points_table['Team'] == team1, 'Points'] += 1
        else:
            points_table = pd.concat([points_table, pd.DataFrame({'Team': [team1], 'Points': [1], 'Score_Difference': 0})], ignore_index=True)

        if team2 in points_table['Team'].values:
            points_table.loc[points_table['Team'] == team2, 'Points'] += 1
        else:
            points_table = pd.concat([points_table, pd.DataFrame({'Team': [team2], 'Points': [1], 'Score_Difference': 0})], ignore_index=True)

# Sort by points, then score difference, then alphabetically for ties
points_table = points_table.sort_values(['Points', 'Score_Difference', 'Team'], ascending=[False, False, True])
points_table = points_table.reset_index(drop=True)
points_table

,Team,Points,Score_Difference
0,SRH,25,658
1,KKR,22,739
2,RR,22,284
3,PBKS,18,532
4,CSK,14,-199
5,GT,10,-201
6,DC,10,-331
7,RCB,10,-343
8,MI,9,-508
9,LSG,8,-631


In [3]:
def simulate_match(team1, team2, points_table):
    score1 = random.randint(50, 300)
    score2 = random.randint(50, 300)
    
    if score1 > score2:
        return team1, score1, score2
    elif score2 > score1:
        return team2, score1, score2
    else:
        team1_index = points_table[points_table['Team'] == team1].index[0]
        team2_index = points_table[points_table['Team'] == team2].index[0]
        if team1_index < team2_index:
            return team1, score1, score2
        else:
            return team2, score1, score2

# Select top 4 teams
top4_teams = points_table.head(4)

# Playoffs
qualifier1_winner, q1_s1, q1_s2 = simulate_match(top4_teams.iloc[0]['Team'], top4_teams.iloc[2]['Team'], points_table)
eliminator_winner, e_s1, e_s2 = simulate_match(top4_teams.iloc[1]['Team'], top4_teams.iloc[3]['Team'], points_table)

qualifier2_teams = [qualifier1_winner, eliminator_winner]
if qualifier1_winner == top4_teams.iloc[0]['Team']:
    qualifier2_loser = top4_teams.iloc[2]['Team']
else:
    qualifier2_loser = top4_teams.iloc[0]['Team']
qualifier2_winner, q2_s1, q2_s2 = simulate_match(qualifier2_loser, eliminator_winner, points_table)

final_winner, f_s1, f_s2 = simulate_match(qualifier1_winner, qualifier2_winner, points_table)


print(f"Qualifier 1: {top4_teams.iloc[0]['Team']} vs {top4_teams.iloc[2]['Team']} -> Winner: {qualifier1_winner} ({q1_s1}-{q1_s2})")
print(f"Eliminator: {top4_teams.iloc[1]['Team']} vs {top4_teams.iloc[3]['Team']} -> Winner: {eliminator_winner} ({e_s1}-{e_s2})")
print(f"Qualifier 2: {qualifier2_loser} vs {eliminator_winner} -> Winner: {qualifier2_winner} ({q2_s1}-{q2_s2})")
print(f"Final: {qualifier1_winner} vs {qualifier2_winner} -> Winner: {final_winner} ({f_s1}-{f_s2})")
print(f"\nThe winner of IPL 2024 is: {final_winner}")

Qualifier 1: SRH vs RR -> Winner: SRH (188-164)
Eliminator: KKR vs PBKS -> Winner: PBKS (93-236)
Qualifier 2: RR vs PBKS -> Winner: RR (177-73)
Final: SRH vs RR -> Winner: SRH (237-180)

The winner of IPL 2024 is: SRH
